# Embeddings

In [1]:
from __future__ import print_function

In [2]:
import tensorflow as tf
import zipfile
import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline

/Users/marcocristo/Library/Enthought/Canopy_64bit/User/lib/python2.7/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')


In [3]:
def reset_graph(seed=42):
    tf.reset_default_graph()
    tf.set_random_seed(seed)
    np.random.seed(seed)

Tradicionalmente, aplicações relacionadas com linguagem natural tratam palavras como eventos discretos. Assim, duas palavras, como _cão_ e _gato_, são representados por identificadores arbitrários que não incorporam qualquer informação sobre as relações existentes entre esses conceitos como, por exemplo, que eles são animais mamíferos, com quatro patas, domesticáveis, etc. Outro problema deste tipo de codificação é que ela induz esparsidade, o que torna complexo o treinamento de um modelo estatístico.

Uma alternativa à representação esparsa arbitrária é a representação com vetores densos. Para obter tais vetores, podemos usar a Hipótese de Distribuição que estabelece que _palavras que aparecem no mesmo contexto, compartilham a mesma semântica_. Há dois tipos gerais de técnicas que se baseiam nessa hipótese: as baseadas em contagem, como a _Indexação Semântica Latente_, e as baseadas em previsão, como os _modelos de linguagem neuro-probabilísticos_. Nesta aula, nos concentramos nas últimas técnicas, que tentam prever diretamente uma palavra alvo apartir da suas vizinhas, aprendendo para isso pequenos vetores densos, que são parâmetros dos modelos. Estes vetores são chamados _embeddings_ de palavras (ou vetores de palavras, na falta de uma melhor tradução para o Português). 

Entre os vários métodos propostos nesta linha, um de grande destaque é o Word2vec (Mikolov et al., http://papers.nips.cc/paper/5021-distributed-representations-of-words-and-phrases-and-their-compositionality.pdf), onde redes neurais rasas e simples são usadas para aprender os embeddings. Com o uso de redes rasas, é possível escalar o modelo para lidar com trilhões de ocorrências de palavras.

Modelos de linguagem neuro-probabilísticos em geral são treinados de acordo com o princípio da máxima verossimelhança. Em particular, o modelo maximiza a probabilidade da próxima palavra, dada as palavras de contexto. Um modelo muito simples, baseado nesta ideia, é ilustrado a seguir.

Nele, há uma única camada escondida de $N$ neurônios. A entrada e a saída são conjuntos de $V$ neurônios, onde $V$ é o tamanho do vocabulário. Os neurônios de entrada representam uma palavra usando codificação _hot vector_. Nos neurônios de saída, a probabilidade de cada palavra é obtida via _softmax_. No exemplo a seguir, dada a frase _the cat_, a palavra _the_ representa a entrada (o contexto) que será usada para prever a palavra-alvo, _cat_.

<img src="images/emb0.png" alt="Exemplo de RNN" style="width: 400px;"/>

Note que como a entrada é 1-hot-vector, se a saída da camada oculta for linear, a saída dessa camada é meramente uma cópia dos pesos da entrada (${\bf h}_w = {\bf x}_w {\bf W}_{I} = {\bf W}_{Iw}$). Ou seja, o código usado para representar cada palavra (${\bf h}_w$) é o peso da entrada corresponde a esta palavra (${\bf W}_{Iw}$). E este será aprendido durante o treino. 

A estimativa da palavra alvo $w_t$ dado contexto é dada pela softmax, ou seja:

$$P(w_t|{\bf h}_w) = softmax(score(w_t, {\bf h}_w)) = \frac{e^{score(w_t, {\bf h}_w)}}{\sum_{w' \in \mathcal{V}}{e^{score(w', {\bf h}_w)}}}$$

onde $\mathcal{V}$ é o vocabulário e _score_ é normalmente o produto interno entre os vetores. Para calcular o erro, basta subtrair o vetor da palavra alvo do vetor de estimativa softmax. Com base no erro, os pesos são atualizados, de forma que a própria representação das palavras (${\bf h}_w$) é moficada para garantir que _palavras que normalmente ocorrem no mesmo contexto sejam capazes de prever umas às outras!_

Observe que o contexto, em geral, é formado por mais de uma palavra (e o alvo não precisa ser necessariamente a próxima). Para isso, basta representar cada palavra na entrada. Abaixo, por exemplo, temos uma arquitetura em que o contexto é formado por _m = 1_ palavras que ocorrem antes e depois da palavra alvo.

<img src="images/cbow.png" alt="Exemplo de RNN" style="width: 400px;"/>

Neste caso, dada a frase _the black cat_, o contexto é formado pelas palavras _the_ e _cat_ e o alvo é _black_. Note que uma camada de projeção aparece no desenho para indicar que as várias entradas discretas serão combinadas para formar uma entrada única: elas podem ser combinadas pela média, soma, multiplicação ou mesmo concatenação. 

Este modelo simples é chamado CBOW (_continuous bag-of-words_). Um modelo alternativo é o _Skip-gram_, que é basicamente a versão invertida do CBOW: ou seja, a entrada corresponde à palavra alvo e o alvo corresponde ao contexto. Embora a diferença entre os modelos pareça arbitrária, o skip-gram tem algumas vantagens em termos de amostragem estatística que o tornam uma solução melhor para o caso de grande número de ocorrência de palavras.

A seguir, vamos estudar esses modelos na prática.

### Obtendo dados

Vamos inicialmente obter palavras de uma amostra de 100Mb a Wikipedia em inglês de 2006.

In [4]:
def fetch_words_data():
    with zipfile.ZipFile('data/wiki2006_100m.zip') as f:
        data = f.read(f.namelist()[0])
    return data.decode("ascii").split()

In [5]:
words = fetch_words_data()

In [8]:
print(' '.join(words[:25]))

anarchism originated as a term of abuse first used against early working class radicals including the diggers of the english revolution and the sans culottes


In [9]:
len(words)

17005207

### Criando um dicionário

In [18]:
from collections import Counter

vocabulary_size = 50000

vocabulary = [("UNK", None)] + Counter(words).most_common(vocabulary_size-1)
vocabulary = np.array([w for w, _ in vocabulary])
dictionary = {w: code for code, w in enumerate(vocabulary)}

In [22]:
dictionary['tiger'], dictionary['anarchism']

(5183, 5239)

In [23]:
data = np.array([dictionary.get(word, 0) for word in words])

In [28]:
print(' '.join(words[:6]))
print(data[:6])
print([vocabulary[id] for id in data[:6]])

anarchism originated as a term of
[5239 3084   12    6  195    2]
[u'anarchism', u'originated', u'as', u'a', u'term', u'of']


### Gerando batches

O modelo que vamos usar é o _skip-gram_. Neste modelo, supondo um contexto de duas palavras, para a sentença _anarchism originated as a term of_, teríamos os seguintes pares de contexto e alvo:

- ([anarchism, as], originated), ([originated, a], as),  ([as, term], a), ([a, of], term), ...

Como no modelo skip-gram nós usamos o alvo para prever o contexto, temos:

- (originated, anarchism), (originated, as), (as, originated), (as, a), (a, as), (a, term), (term, a), (term, of) ...

A seguir vamos definir uma função (generate_batch) que obtem exemplos para treinar um modelo _skip gram_. 

In [29]:
import random
from collections import deque

# num_skips = How many times to reuse an input to generate a label
# skip_window = How many words to consider left and right.
def generate_batch(batch_size, num_skips, skip_window):
    global data_index
    assert batch_size % num_skips == 0
    assert num_skips <= 2 * skip_window
    batch = np.ndarray(shape=(batch_size), dtype=np.int32)
    labels = np.ndarray(shape=(batch_size, 1), dtype=np.int32)
    span = 2 * skip_window + 1 # [ skip_window target skip_window ]
    buffer = deque(maxlen=span)
    for _ in range(span):
        buffer.append(data[data_index])
        data_index = (data_index + 1) % len(data)
    for i in range(batch_size // num_skips):
        target = skip_window  # target label at the center of the buffer
        targets_to_avoid = [ skip_window ]
        for j in range(num_skips):
            while target in targets_to_avoid:
                target = random.randint(0, span - 1)
            targets_to_avoid.append(target)
            batch[i * num_skips + j] = buffer[skip_window]
            labels[i * num_skips + j, 0] = buffer[target]
        buffer.append(data[data_index])
        data_index = (data_index + 1) % len(data)
    return batch, labels

In [31]:
data_index = 0
batch, labels = generate_batch(8, 2, 1)
print(batch.shape)
print(labels.shape)

(8,)
(8, 1)


In [32]:
batch

array([3084, 3084,   12,   12,    6,    6,  195,  195], dtype=int32)

In [35]:
for i in range(len(batch)):
    print('%d %10s -> %s' % (i, 
                             vocabulary[batch[i]], 
                             vocabulary[labels[:,0][i]]))

0 originated -> anarchism
1 originated -> as
2         as -> a
3         as -> originated
4          a -> term
5          a -> as
6       term -> a
7       term -> of


### Skip-gram

In [36]:
batch_size = 128
embedding_size = 128  # Dimension of the embedding vector.
skip_window = 1       # How many words to consider left and right.
num_skips = 2         # How many times to reuse an input to generate a label.

# We pick a random validation set to sample nearest neighbors. Here we limit the
# validation samples to the words that have a low numeric ID, which by
# construction are also the most frequent.
valid_size = 16     # Random set of words to evaluate similarity on.
valid_window = 100  # Only pick dev samples in the head of the distribution.
valid_examples = np.random.choice(valid_window, valid_size, replace=False)
num_sampled = 64    # Number of negative examples to sample.

learning_rate = 0.01

In [ ]:
reset_graph()

# Input data.
train_labels = tf.placeholder(tf.int32, shape=[batch_size, 1])
valid_dataset = tf.constant(valid_examples, dtype=tf.int32)

Note que maximizar o $\log P(w_t|{\bf h}_w)$ corresponde a maximizar a função de custo:

$$J = \log \frac{e^{score(w_t, {\bf h}_w)}}{\sum_{w' \in \mathcal{V}}{e^{score(w', {\bf h}_w)}}} = score(w_t, {\bf h}_w) - \log{\sum_{w' \in \mathcal{V}}{e^{score(w', {\bf h}_w)}}}$$

Enquanto $J$ corresponde a uma estimativa adequada de um ponto de vista probabilístico, ela é muito cara para ser processada para cada instância de treino, uma vez que o somatório é marginalizado sobre todas as palavras do vocabulário ($w' \in \mathcal{V}$).

Assim, em lugar de calcular um modelo probabilístico completo, nosso objetivo no treino será o de uma regressão logística binária. Em particular, para nosso modelo skip-gram, queremos discriminar quem é o melhor previsor para as palavras de contexto ${\bf h}$: o alvo $w_t$ ou $k$ palavras tomadas aleatoriamente (ruído ${\bf \tilde{w}}$)?

Formalmente, nosso objetivo passa a ser maximizar:

$$J_{NEG} = \log Q_{\theta}(D=1 | w_t, {\bf h}) + k \mathop{\mathbb{E}}_{{\bf \tilde{w}} \approx P_{noise}}[\log ⁡Q_{\theta} (D=0 | {\bf \tilde{w}}, {\bf h})]$$

onde $\log Q_{\theta}(D=1 | w_t, {\bf h})$ é a probabilidade (obtida com um regressor logístico binário) de $w_t$ ser vista no contexto ${\bf h}$ para um conjunto de dados $D$, considerando um conjunto de embeddings $\theta$. O segundo componente da equação corresponde à probabilidade de que $k$ palavras contrastivas ${\bf \tilde{w}}$ (tomadas aleatorimente de $\mathcal{V}$) _não_ sejam vistas no contexto ${\bf h}$. Por exemplo, digamos que temos a frase _the black cat_, com contexto dado por (_the_, _cat_) e alvo dado por _cat_. Em $J_{NEG}$ pretende-se maximizar a probabilidade de _black_ ser vista no contexto (_the_, _cat_) ao mesmo tempo que se minimiza a probabilidade de que $k$ palavras tomadas aleatoriamente, como (_then_, _observe_, _those_) para $k=3$, sejam vistas no mesmo contexto. A figura abaixo ilustra essa estratégia em um modelo skip-gram:

<img src="images/skip-gram-neg-sampling.png" alt="Relações em embeddings" style="width: 500px;"/>

Em suma, o objetivo é máximizado quando o modelo associa probabilidades altas para palavras que fazem sentido no contexto (_black_) e baixa para palavras ruído (_then_, _observe_, _those_). Esta estratégia é chamada _Negative Sampling_ e há duas boas razões para usá-la: (1) as atualizações que ela propões aproximam as atualizações da softmax no limite; e (2) o custo por instância de treino do estimador cai de $O(\mathcal{V})$ para $O(k)$, $k << \mathcal{V}$. O tensorflow oferece um estimador baseado nesta ideia, o _noise-contrastive estimation (NCE) loss_ -- `tf.nn.nce_loss()`.

In [ ]:
vocabulary_size = 50000
embedding_size = 150

# Look up embeddings for inputs.
init_embeds = tf.random_uniform([vocabulary_size, embedding_size], -1.0, 1.0)
embeddings = tf.Variable(init_embeds)

In [ ]:
train_inputs = tf.placeholder(tf.int32, shape=[None])
embed = tf.nn.embedding_lookup(embeddings, train_inputs)

In [ ]:
# Construct the variables for the NCE loss
nce_weights = tf.Variable(
    tf.truncated_normal([vocabulary_size, embedding_size],
                        stddev=1.0 / np.sqrt(embedding_size)))
nce_biases = tf.Variable(tf.zeros([vocabulary_size]))

# Compute the average NCE loss for the batch.
# tf.nce_loss automatically draws a new sample of the negative labels each
# time we evaluate the loss.
loss = tf.reduce_mean(
    tf.nn.nce_loss(nce_weights, nce_biases, train_labels, embed,
                   num_sampled, vocabulary_size))

# Construct the Adam optimizer
optimizer = tf.train.AdamOptimizer(learning_rate)
training_op = optimizer.minimize(loss)

# Compute the cosine similarity between minibatch examples and all embeddings.
norm = tf.sqrt(tf.reduce_sum(tf.square(embeddings), axis=1, keep_dims=True))
normalized_embeddings = embeddings / norm
valid_embeddings = tf.nn.embedding_lookup(normalized_embeddings, valid_dataset)
similarity = tf.matmul(valid_embeddings, normalized_embeddings, transpose_b=True)

# Add variable initializer.
init = tf.global_variables_initializer()

### Treinamento

In [ ]:
def plot_with_labels(low_dim_embs, labels):
    assert low_dim_embs.shape[0] >= len(labels), "More labels than embeddings"
    plt.figure(figsize=(18, 18))  #in inches
    for i, label in enumerate(labels):
        x, y = low_dim_embs[i,:]
        plt.scatter(x, y)
        plt.annotate(label,
                     xy=(x, y),
                     xytext=(5, 2),
                     textcoords='offset points',
                     ha='right',
                     va='bottom')

In [ ]:
from sklearn.manifold import TSNE

tsne = TSNE(perplexity=30, n_components=2, init='pca', n_iter=5000)
plot_only = 500
low_dim_embs = tsne.fit_transform(final_embeddings[:plot_only,:])
labels = [vocabulary[i] for i in range(plot_only)]
plot_with_labels(low_dim_embs, labels)

Como treinamos pouco nossos embeddings, muitas das suas propriedades não estão ainda observáveis. Contudo, ao analisar um conjunto de embeddings bem treinado, é fácil perceber que os embeddings capturam informação semântica geral e bastante útil. De fato, o uso de embeddings melhora o desempenho de várias atividades envolvendo linguagem natural, tais como agrupamento, classificação, reconhecimento de tipos de discurso, etc.

### Usando _embeddings_ Pré-treinados (com Gensim)

Especializações comumente observadas nos eixos dos embeddings incluem relacionamentos semânticos como o de gênero (feminino v. masculino), conjugações verbais, relações geográficas, etc. Algumas destas são demonstradas nas figuras abaixo:

<img src="images/linear-relationships.png" alt="Relações em embeddings" style="width: 600px;"/>

Podemos observar tais especializações em conjuntos pré-treinados de embeddings. Há vários destes conjuntos disponíveis:

* Inglês: https://github.com/alexandres/lexvec#pre-trained-vectors
* Inlgês (Glove): https://nlp.stanford.edu/projects/glove/
* Português: http://www.nilc.icmc.usp.br/nilc/index.php/repositorio-de-word-embeddings-do-nilc

Nos exemplos a seguir, usamos um conjunto de embeddings para língua portuguesa, obtidos do NILC, treinados com skip-gram. Dos 2.6 Gbytes de dados da coleção original, separamos cerca de 24 mil embeddings, usando uma lista de palavras da língua portuguesa.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
def load_embeddings(fname):
    vocab = []
    embd = []
    file = open(fname,'r')
    for line in file.readlines():
        row = line.strip().split(' ')
        if len(row) > 2:
            vocab.append(row[0])
            embd.append(row[1:])
    print('done: %d %d-D embeddings loaded!' % (len(vocab), len(embd[0])))
    file.close()
    return vocab, embd

In [ ]:
fname = 'data/subset_skip_s300.txt'
vocabulary, embd = load_embeddings(fname)
vocab_size = len(vocabulary)
embedding_dim = len(embd[0])
final_embeddings = np.asarray(embd)

O material desta aula se baseou no tutorial de Word2vec do tensorflow (https://www.tensorflow.org/tutorials/word2vec) e no livro de Aurélien Géron, "Hands-on Machine Learning with Skitlearn and Tensorflow".